In [4]:
import requests
import json
from pathlib import Path

There must be a file named 'token.txt' with your Zenodo API access token (see https://developers.zenodo.org)

In [14]:
with open(Path('.')  / "token.txt", 'r') as f:
    ACCESS_TOKEN = f.readlines()[0].strip()

In [15]:
def zenodo_query(query, from_cache=True):
    cache_file = Path('.') / 'data' / f'{query}.json'
    
    # if `from_cache` is True but cache file doesn't exist, then set `from_cache` to False 
    from_cache = from_cache and cache_file.exists()

    if not from_cache:
        response = requests.get('https://zenodo.org/api/records',
                    params={'q': query},
                    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'})
        data = response.json()
        with open(cache_file, 'w') as f:
            json.dump(data, f)

    with open(cache_file, 'r') as f:
        return json.load(f)

In [13]:
portal_names = ['ontoportal', 'bioportal', 'agroportal', 'ecoportal', 'earthportal', 'biodivportal']

for name in portal_names:
    _ = zenodo_query(name, from_cache=True)
    

In [16]:
def records():
    results = dict()
    for name in portal_names:
        data = zenodo_query(name)
        for rec in data['hits']['hits']:
            if 'communities' not in rec['metadata'].keys():
                rec['metadata']['communities'] = []
            results[rec['doi']] = {
                'url': rec['doi_url'],
                'authors': [item['name'] for item in rec['metadata']['creators']],
                'title': rec['title'], 
                'owners': rec['owners'],
                'communities': [item['id'] for item in rec['metadata']['communities']]
            }
    return results

def records_in_ontoportal_community():
    data = records()
    results = dict()
    for k in data.keys(): 
        if 'ontoportal' in data[k]['communities']:
            results[k] = data[k]
    return results

def records_not_in_ontoportal_community():
    data = records()
    results = dict()
    for k in data.keys(): 
        if 'ontoportal' not in data[k]['communities']:
            results[k] = data[k]
    return results

def record_record_list():
    #from IPython.display import display_markdown
    import datetime
    md = f"""OntoPortal records in Zenodo
============================

Tickmarks are telling if the record is in the *ontoportal* community.

Keywords used for building the list: *{'*, *'.join(portal_names)}*

Last update: {datetime.datetime.now()}

---
"""
    data = records()
    for doi, rec in data.items():
        md+=f""" - [{'x' if 'ontoportal' in rec['communities'] else ' '}] [{doi}]({rec['url']}): {rec['authors'][0]}, {rec['title']}
"""
    #display_markdown(md, raw=True)
    print(md)

def display_record_table():
    from IPython.display import display_markdown 
    md = """
|     | doi | 1st author | title |
|-----|-----|------------|-------|
"""
    data = records()
    for doi, rec in data.items():
        md+=f"""| {'x' if 'ontoportal' in rec['communities'] else ' '} | [{doi}]({rec['url']}) | {rec['authors'][0]} | {rec['title']} |
"""
    display_markdown(md, raw=True)

In [17]:
record_record_list()

OntoPortal records in Zenodo

Tickmarks are telling if the record is in the *ontoportal* community.

Keywords used for building the list: *ontoportal*, *bioportal*, *agroportal*, *ecoportal*, *earthportal*, *biodivportal*

Last update: 2025-11-11 20:27:57.724709

---
 - [ ] [10.5281/zenodo.10020455](https://doi.org/10.5281/zenodo.10020455): Jonquet, Clement, Ontology Repositories and Semantic Artefact Catalogues with the OntoPortal Technology
 - [ ] [10.5281/zenodo.12682450](https://doi.org/10.5281/zenodo.12682450): Redondo Aniorte, Francisco Javier, OntoPortal ontologies
 - [x] [10.5281/zenodo.15863671](https://doi.org/10.5281/zenodo.15863671): Cecconi, Baptiste, OPAL - Ontology Portal for Astronomy Linked-data
 - [ ] [10.5281/zenodo.13983396](https://doi.org/10.5281/zenodo.13983396): Tykhonov, Vyacheslav, External controlled vocabularies support in the European Open Science Cloud (EOSC)
 - [ ] [10.5281/zenodo.13143176](https://doi.org/10.5281/zenodo.13143176): Baum, Roman, OLS, OntoP

In [18]:
display_record_table()


|     | doi | 1st author | title |
|-----|-----|------------|-------|
|   | [10.5281/zenodo.10020455](https://doi.org/10.5281/zenodo.10020455) | Jonquet, Clement | Ontology Repositories and Semantic Artefact Catalogues with the OntoPortal Technology |
|   | [10.5281/zenodo.12682450](https://doi.org/10.5281/zenodo.12682450) | Redondo Aniorte, Francisco Javier | OntoPortal ontologies |
| x | [10.5281/zenodo.15863671](https://doi.org/10.5281/zenodo.15863671) | Cecconi, Baptiste | OPAL - Ontology Portal for Astronomy Linked-data |
|   | [10.5281/zenodo.13983396](https://doi.org/10.5281/zenodo.13983396) | Tykhonov, Vyacheslav | External controlled vocabularies support in the European Open Science Cloud (EOSC) |
|   | [10.5281/zenodo.13143176](https://doi.org/10.5281/zenodo.13143176) | Baum, Roman | OLS, OntoPortal, Skosmos - API mapping specifications for API Gateway and Terminology Service Suite |
|   | [10.5281/zenodo.17222813](https://doi.org/10.5281/zenodo.17222813) | Voß, Jakob | Integration of mapping tool Cocoda with OntoPortal |
|   | [10.5281/zenodo.15056529](https://doi.org/10.5281/zenodo.15056529) | Jonquet, Clement | FAIR-IMPACT - Advancing FAIR Semantic Artefact Discovery and Search: OntoPortal Federation Joint Release |
|   | [10.5281/zenodo.14181064](https://doi.org/10.5281/zenodo.14181064) | Baum, Roman | First steps of the basic service Terminology Services 4 NFDI - Step by step through the initialization to the integration phase |
|   | [10.5281/zenodo.13786177](https://doi.org/10.5281/zenodo.13786177) | Dillo, Ingrid | FAIR-IMPACT National Roadshow France: French data repositories and their use of semantics |
|   | [10.5281/zenodo.13837021](https://doi.org/10.5281/zenodo.13837021) | Kokkinaki, Alexandra | FAIRness assessment of  vocabularies: Insights from  the British Oceanographic  Data Centre team |
|   | [10.5281/zenodo.15176099](https://doi.org/10.5281/zenodo.15176099) | Baum, Roman | Technical Architecture of Terminology Services 4 NFDI (TS4NFDI) |
|   | [10.5281/zenodo.10550341](https://doi.org/10.5281/zenodo.10550341) | Baum, Roman | Terminology Services for NFDI (TS4NFDI) - Unleashing the potential of service applications |
|   | [10.5281/zenodo.14643279](https://doi.org/10.5281/zenodo.14643279) | Jonquet, Clement | D4.2 - FAIR semantic artefact lifecycle from engineering, to sharing |
|   | [10.5281/zenodo.13862158](https://doi.org/10.5281/zenodo.13862158) | Baum, Roman | TS4NFDI Requirements Analysis Report |
|   | [10.5281/zenodo.14191078](https://doi.org/10.5281/zenodo.14191078) | Kihal, Bilel | Leveraging AgroPortal ontologies to  ease metadata completion and data  discovery in Data INRAE |
|   | [10.5281/zenodo.15760162](https://doi.org/10.5281/zenodo.15760162) | Jennifer Vendetti | ncbo/owlapi_wrapper: owlapi_wrapper 1.5.0 |
|   | [10.5281/zenodo.12799796](https://doi.org/10.5281/zenodo.12799796) | Jonquet, Clement | M4.4 - Review of Semantic Artefact Catalogues and guidelines for serving FAIR semantic artefacts in EOSC |
|   | [10.5281/zenodo.14230727](https://doi.org/10.5281/zenodo.14230727) | Shigapov, Renat | A terminology service via TS4NFDI for BERD@NFDI knowledge graph infrastructure |
|   | [10.5281/zenodo.10684437](https://doi.org/10.5281/zenodo.10684437) | Clement Jonquet | PREreview of "DRAFT: Developing and implementing the semantic interoperability recommendations of the EOSC Interoperability Framework" |
|   | [10.3897/aca.8.e151393](https://doi.org/10.3897/aca.8.e151393) | Pierkot, Christelle | Enhancing Semantic Interoperability for Land Surface Data: The Role of EarthPortal |
|   | [10.5281/zenodo.167619](https://doi.org/10.5281/zenodo.167619) | Graybeal, John | Increasing NCBO BioPortal and CEDAR Synergy for BD2K |
|   | [10.5281/zenodo.15667](https://doi.org/10.5281/zenodo.15667) | Matentzoglu, Nicolas | BioPortal Snapshot 27.01.2015 |
|   | [10.5281/zenodo.10842](https://doi.org/10.5281/zenodo.10842) | Matentzoglu, Nicolas | BioPortal Snapshot 30.06.2014 |
|   | [10.5281/zenodo.439510](https://doi.org/10.5281/zenodo.439510) | Matentzoglu, Nicolas | BioPortal Snapshot 30.03.2017 |
|   | [10.5281/zenodo.14188134](https://doi.org/10.5281/zenodo.14188134) | Harris, Nomi | Data and slides from May 2023 virtual workshop on change languages in ontologies and knowledge graphs |
|   | [10.5281/zenodo.5733820](https://doi.org/10.5281/zenodo.5733820) | Liu, Andrew W. | Job files for automated phenol-chloroform extraction (RBI LabDroid Maholo) |
|   | [10.5281/zenodo.8325085](https://doi.org/10.5281/zenodo.8325085) | Lam, An Ngoc | A Performance Evaluation of OWL 2 DL Reasoners using ORE 2015 and Very Large Bio Ontologies |
|   | [10.5281/zenodo.10624996](https://doi.org/10.5281/zenodo.10624996) | Leo, María | The drivers of plant turnover change across spatial scales in the Azores - dataset associated |
|   | [10.5281/zenodo.15284917](https://doi.org/10.5281/zenodo.15284917) | Vendetti, Jennifer | BioPortal Software for NAR-00680-Web-S-2025.R1 Publication |
|   | [10.5281/zenodo.10475934](https://doi.org/10.5281/zenodo.10475934) | Lam, An Ngoc | A Performance Evaluation of OWL 2 DL Reasoners using ORE 2015 and Very Large Bio Ontologies |
|   | [10.5281/zenodo.31621](https://doi.org/10.5281/zenodo.31621) | Nicolas Matentzoglu | Module-based classification of OWL ontologies - Experiment 3 Dataset (Subsumption Test Hardness Survey BioPortal) |
|   | [10.5281/zenodo.6564563](https://doi.org/10.5281/zenodo.6564563) | Hoffman, Leon | Talassia coriacea |
|   | [10.5281/zenodo.5227312](https://doi.org/10.5281/zenodo.5227312) | Eamonn Maguire | ISA-tools/OntoMaton: Ontomaton Google Spreadsheets Add-On version 4.0 - Upgrade |
|   | [10.5281/zenodo.819726](https://doi.org/10.5281/zenodo.819726) | Xin He | statbio/OntoSuite-Miner: OntoSuite-Miner v1.0 |
|   | [10.35940/ijrte.F5327.039621](https://doi.org/10.35940/ijrte.F5327.039621) | R. Jeena | A Novel Approach for Healthcare Information System using Cloud |
|   | [10.5281/zenodo.15284288](https://doi.org/10.5281/zenodo.15284288) | Michael Dorf | ncbo/ncbo_ontology_recommender: v5.33.4 for Nucleic Acids Research |
|   | [10.5281/zenodo.15284304](https://doi.org/10.5281/zenodo.15284304) | Michael Dorf | ncbo/ncbo_annotator: v5.33.4 for Nucleic Acids Research |
|   | [10.5281/zenodo.16778203](https://doi.org/10.5281/zenodo.16778203) | Adam SL Graefe | BIH-CEI/rd-cdm: v2.0.2 |
|   | [10.5281/zenodo.4247134](https://doi.org/10.5281/zenodo.4247134) | Wimmics Research Team | Covid-on-the-Web dataset |
|   | [10.5281/zenodo.14884034](https://doi.org/10.5281/zenodo.14884034) | Alharbi, Reham | Exploring the Role of Generative AI in Constructing Knowledge Graphs for Drug Indications with Medical Context |
|   | [10.5061/dryad.vq587](https://doi.org/10.5061/dryad.vq587) | Bertone, Matthew A. | Data from: Matching arthropod anatomy ontologies to the Hymenoptera Anatomy Ontology: results from a manual alignment |
|   | [10.11646/phytotaxa.633.1.9](https://doi.org/10.11646/phytotaxa.633.1.9) | Melo, Aline | Typifications in Piper consanguineum (Piperaceae) |
|   | [10.5281/zenodo.15682](https://doi.org/10.5281/zenodo.15682) | Matentzoglu, Nicolas | Module Data for the Empirical Investigation of Subsumption Test Hardness in Description Logic Classification |
|   | [10.5281/zenodo.14651571](https://doi.org/10.5281/zenodo.14651571) | Lin, Zhengyu | denovoFAIRificationworkflowtestdataset_CLI_materialize_output |
|   | [10.5281/zenodo.6941350](https://doi.org/10.5281/zenodo.6941350) | Taneja, Sanya Bathla | Evaluation of Named Entity Recognition Systems to Improve Ontology Concept Annotation for Biomedical Knowledge Graphs |
|   | [10.5281/zenodo.15262965](https://doi.org/10.5281/zenodo.15262965) | Aubin, Sophie | FAIR-IMPACT AHM - T4.5 Use cases demonstration session |
|   | [10.5281/zenodo.14192645](https://doi.org/10.5281/zenodo.14192645) | Tireau, Anne | Enabling interoperability between  AgroPortal and PHIS information system  data repository for enhanced phenomics  data annotation and exchange |
|   | [10.5281/zenodo.13135055](https://doi.org/10.5281/zenodo.13135055) | Vianello, Andrea | FAIRness assessment of datasets and ontologies at Eurac Research |
|   | [10.5281/zenodo.7937430](https://doi.org/10.5281/zenodo.7937430) | Allyson L. Lister | PREreview of "A maturity model for catalogues of semantic artefacts" |
|   | [10.5281/zenodo.16793001](https://doi.org/10.5281/zenodo.16793001) | Barro, A. | Comparison of beef eating quality terminology databases |
|   | [10.5281/zenodo.12579779](https://doi.org/10.5281/zenodo.12579779) | Wilson, Antony | D4.3 - Specification of shared metadata description of semantic artefacts and their catalogues including common reference API |
|   | [10.5281/zenodo.4885364](https://doi.org/10.5281/zenodo.4885364) | Larysa O. Ligonenko | Digitalization of the agricultural sphere: state, problems and prospects |
|   | [10.5281/zenodo.3520300](https://doi.org/10.5281/zenodo.3520300) | Sophie Aubin | D2KAB project taking off: Data to Knowledge in Agronomy and Biodiversity |
|   | [10.21303/2504-5695.2021.002075](https://doi.org/10.21303/2504-5695.2021.002075) | Chynchyk, Oleksandr | Biological nitrogen in increasing the productivity of beans (grains) |
|   | [10.5281/zenodo.17383236](https://doi.org/10.5281/zenodo.17383236) | Ivanov, Roman | Theoretical substantiation of the system of factors influencing the agricultural development: the Ukrainian context |
|   | [10.3897/BDJ.12.e128585](https://doi.org/10.3897/BDJ.12.e128585) | Deans, Andrew | GallOnt: An ontology for plant gall phenotypes |
|   | [10.5281/zenodo.1111532](https://doi.org/10.5281/zenodo.1111532) | Saif Obaid Alkaabi | Factors Affecting M-Government Deployment and Adoption |
|   | [10.5281/zenodo.1128004](https://doi.org/10.5281/zenodo.1128004) | Chaimaa Messaoudi | A Survey of Semantic Integration Approaches in Bioinformatics |
|   | [10.5281/zenodo.14186722](https://doi.org/10.5281/zenodo.14186722) | Rosati, Ilaria | Improving ecological (meta)data FAIRness through semantic services: integration of EcoPortal in LifeWatch Italy new platforms |
|   | [10.5281/zenodo.12189309](https://doi.org/10.5281/zenodo.12189309) | Romero, Fátima | Pterophoridae Latreille 1802 |
|   | [10.5281/zenodo.8029700](https://doi.org/10.5281/zenodo.8029700) | Ustjuzhanin, Petr | Emmelina doroshkini Ustjuzhanin & Kovtunovich 2021 |
|   | [10.5281/zenodo.8029704](https://doi.org/10.5281/zenodo.8029704) | Ustjuzhanin, Petr | Adaina zephyria Barnes & Lindsey 1921 |
|   | [10.5281/zenodo.14222411](https://doi.org/10.5281/zenodo.14222411) | Fiore, Nicola | PIDs as a cornerstone in actualising the FAIR principles within the LifeWatch infrastructure |
|   | [10.37828/em.2022.58.3](https://doi.org/10.37828/em.2022.58.3) | Ustjuzhanin, Petr | New species of plume moths (Lepidoptera: Pterophoridae) from Argentina |
|   | [10.30550/j.azl/2022.66.2/2022-07-25](https://doi.org/10.30550/j.azl/2022.66.2/2022-07-25) | Romero, Fátima | Lepidópteros acuáticos y semiacuáticos en Argentina: lista actualizada, distribución y hábitos de vida |
|   | [10.5281/zenodo.14186508](https://doi.org/10.5281/zenodo.14186508) | Pierkot, Christelle | Enhance the semantic functionally of the national Earth & Environmental Data Repository by integrating it with the EarthPortal |
|   | [10.5281/zenodo.15074471](https://doi.org/10.5281/zenodo.15074471) | Karam, Naouel | Towards the NFDI4Biodiversity Knowledge Graph: Trait Data as a Use Case |
|   | [10.5281/zenodo.14882871](https://doi.org/10.5281/zenodo.14882871) | Rehfeld, Rica | How to use? Tools und Services des NFDI4Biodiversity Konsortiums |
|   | [10.5281/zenodo.13134543](https://doi.org/10.5281/zenodo.13134543) | Karam, Naouel | Enabling Semantic Search and Interoperability within NFDI4Biodiversity using BiodivPortal |
